# Read SIFT 1 million 

In [1]:
path = joinpath(homedir(), "Datasets", "SIFT1M",
    "sift-128-euclidean.hdf5")

"/Users/davidbuchaca/Datasets/SIFT1M/sift-128-euclidean.hdf5"

In [2]:
using HDF5

In [3]:
f = h5open(path, "r")

🗂️ HDF5.File: (read-only) /Users/davidbuchaca/Datasets/SIFT1M/sift-128-euclidean.hdf5
├─ 🏷️ distance
├─ 🔢 distances
├─ 🔢 neighbors
├─ 🔢 test
└─ 🔢 train

In [4]:
X_tr_vecs = read(f["train"])
X_te_vecs = read(f["test"]);
neighbors = read(f["neighbors"])
distances = read(f["distances"])

@show size(X_tr_vecs)
@show size(X_te_vecs)
@show size(neighbors)
@show size(distances)

size(X_tr_vecs) = (128, 1000000)
size(X_te_vecs) = (128, 10000)
size(neighbors) = (100, 10000)
size(distances) = (100, 10000)


(100, 10000)

### Mean Squared Error

In [5]:
function MSE(X, query)
    d = (query .- X) .* (query .- X)
    res = d /length(query);
    return sum(res, dims=1)
end

MSE (generic function with 1 method)

In [6]:
function MSE_2(X, query)
    n_features, n_examples = size(X)
    result = zeros(n_examples)
    for m in 1:n_examples
        res = zero(eltype(X))
        for j in 1:n_features
            aux = (query[j] .- X[j,m])
            res += aux * aux
        end
        result[m] = res/n_features
    end
    return result
end

MSE_2 (generic function with 1 method)

In [7]:
query = X_te_vecs[:,1];

In [8]:
using BenchmarkTools

In [9]:
@benchmark MSE(X_te_vecs, query)

BenchmarkTools.Trial: 677 samples with 1 evaluation.
 Range (min … max):  2.326 ms … 42.588 ms  ┊ GC (min … max):  0.00% … 82.58%
 Time  (median):     6.954 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   7.367 ms ±  6.235 ms  ┊ GC (mean ± σ):  15.24% ± 15.16%

  ▃▅   ▄▆█▂                                                   
  ██▇▆▇████▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▁▄▁▁▄▆▅▆ ▇
  2.33 ms      Histogram: log(frequency) by time     40.6 ms <

 Memory estimate: 9.80 MiB, allocs estimate: 6.

In [10]:
@benchmark MSE_2(X_te_vecs, query)

BenchmarkTools.Trial: 3317 samples with 1 evaluation.
 Range (min … max):  1.364 ms …   2.351 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.454 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.491 ms ± 109.394 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▁▅▆█▇▅▆▅▃▁▁                                                 
  ▄███████████▇█▇▆▇▆▇▅▅▅▅▅▅▅▅▅▅▅▅▅▄▅▄▄▄▃▃▄▃▃▃▃▃▃▂▃▂▂▃▂▂▂▂▃▂▁▂ ▄
  1.36 ms         Histogram: frequency by time        1.85 ms <

 Memory estimate: 78.20 KiB, allocs estimate: 2.

In [11]:
@benchmark MSE(X_tr_vecs, query)

BenchmarkTools.Trial: 7 samples with 1 evaluation.
 Range (min … max):  673.161 ms … 895.941 ms  ┊ GC (min … max):  0.20% … 16.72%
 Time  (median):     821.614 ms               ┊ GC (median):     9.06%
 Time  (mean ± σ):   819.089 ms ±  73.914 ms  ┊ GC (mean ± σ):  10.33% ±  5.52%

  ▁                                    ▁ ▁▁▁                  █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁███▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  673 ms           Histogram: frequency by time          896 ms <

 Memory estimate: 980.38 MiB, allocs estimate: 6.

In [12]:
@benchmark MSE_2(X_tr_vecs, query)

BenchmarkTools.Trial: 32 samples with 1 evaluation.
 Range (min … max):  144.341 ms … 164.304 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     157.739 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   156.720 ms ±   4.986 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁     ▁  ▁  ▁   ▁        ▁    ▁▁▁▁█ ▁▁ ▁▁▁█ ▁▁█ ▁ █▁ █ █    ▁  
  █▁▁▁▁▁█▁▁█▁▁█▁▁▁█▁▁▁▁▁▁▁▁█▁▁▁▁█████▁██▁████▁███▁█▁██▁█▁█▁▁▁▁█ ▁
  144 ms           Histogram: frequency by time          164 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 2.

### Finding top k distances (and their ids)


The first naive thing we can do consist on computing all distances and then sorting them to get the top k closest vectors to the query vector

In [13]:
function top_k_ids(X, query)
    distances = MSE_2(X, query)
    top_k_indices = sortperm(distances)
    return top_k_indices
end

top_k_ids (generic function with 1 method)

In [14]:
@benchmark top_k_ids(X_te_vecs, query)[1:10]

BenchmarkTools.Trial: 2131 samples with 1 evaluation.
 Range (min … max):  2.001 ms …  40.367 ms  ┊ GC (min … max): 0.00% … 94.28%
 Time  (median):     2.311 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.332 ms ± 837.968 μs  ┊ GC (mean ± σ):  0.77% ±  2.04%

     ▃          ▁ ▁▁▂▄▄▄▆▇▅▇█▆▅▅▆▂▄▅▄▄▂▂ ▁▁                    
  ▃▄██▇▆▄▆▃▆▄▅▇▆█▇██████████████████████████▆█▆▅▄▄▅▄▃▃▃▃▃▂▂▂▂ ▅
  2 ms            Histogram: frequency by time        2.68 ms <

 Memory estimate: 156.58 KiB, allocs estimate: 6.

A slightly better approach consist on using `partialsortperm` to simply sort a subset of the distances vector.

In [15]:
function top_k_ids_2(X, query, k)
    distances = MSE_2(X, query)
    top_k_indices = partialsortperm(distances, 1:k)
    return top_k_indices
end

top_k_ids_2 (generic function with 1 method)

In [16]:
@benchmark top_k_ids_2(X_te_vecs, query, 10)

BenchmarkTools.Trial: 2941 samples with 1 evaluation.
 Range (min … max):  1.367 ms …  41.973 ms  ┊ GC (min … max): 0.00% … 95.87%
 Time  (median):     1.676 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.688 ms ± 756.365 μs  ┊ GC (mean ± σ):  0.81% ±  1.77%

        ▃▅▂▂      ▁▂▁ ▂▆▆▅▄▂▄▇▆▆▅▆█▅▃▂▂ ▁                      
  ▁▁▁▁▁▄██████▆▅▅▇███████████████████████▆██▇▅▅▄▅▃▅▂▃▃▃▂▃▂▂▁▂ ▅
  1.37 ms         Histogram: frequency by time        2.04 ms <

 Memory estimate: 156.58 KiB, allocs estimate: 9.

### Storing top k distances in a priority queue

A better alternative consists on using a priority queue. This. queue will keep only k distances in memory (no need to store all distances between the query point and all possible candidates).

In [17]:
sort!([1,54,3,24,10])

5-element Vector{Int64}:
  1
  3
 10
 24
 54

In [18]:
a = [1,2,3,4,5]

5-element Vector{Int64}:
 1
 2
 3
 4
 5

In [19]:
using BenchmarkTools

In [20]:
function MSE_3(X, query, top_k)
    result = sort(MSE_2(X[:, 1:top_k], query))
    n_features, n_examples = size(X)
    
    for m in top_k:n_examples
        res = zero(eltype(X))
        for j in 1:n_features
            aux = (query[j] .- X[j,m])
            res += aux * aux
        end
        dist = res/n_features
        
        # see if current mse is in the top pile
        if dist < result[end]
            j = top_k 
            #revisamos la lista resultado de izq a derecha
            while dist < result[j-1]
                j = j-1
                if j == 1
                    break
                end
            end
            
            result[j+1:end] .= result[j:end-1]
            result[j] = dist
         end

    end
    return result
end

MSE_3 (generic function with 1 method)

In [21]:
@benchmark MSE_3(X_tr_vecs, query, 10) 

BenchmarkTools.Trial: 32 samples with 1 evaluation.
 Range (min … max):  150.011 ms … 163.876 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     157.522 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   157.223 ms ±   3.915 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

       █                    ▃                 ▃ ▃  ▃   ▃         
  ▇▁▁▁▁█▁▁▇▇▇▁▁▁▁▁▇▁▁▇▁▇▁▁▁▁█▁▇▇▇▇▁▁▇▁▇▁▁▇▇▇▇▁█▁█▁▁█▁▁▁█▁▇▁▁▁▁▇ ▁
  150 ms           Histogram: frequency by time          164 ms <

 Memory estimate: 18.64 KiB, allocs estimate: 114.

In [22]:
@benchmark top_k_ids_2(X_tr_vecs, query, 10)

BenchmarkTools.Trial: 29 samples with 1 evaluation.
 Range (min … max):  162.555 ms … 204.958 ms  ┊ GC (min … max): 0.00% … 15.07%
 Time  (median):     173.513 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   173.905 ms ±   7.639 ms  ┊ GC (mean ± σ):  0.61% ±  2.80%

            ▂    █   ▂  ▂                                        
  ▅▁▁▅▅▅▁▅▁▅█▅█▅▁█▁▅▁█▁▅█▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅ ▁
  163 ms           Histogram: frequency by time          205 ms <

 Memory estimate: 15.26 MiB, allocs estimate: 9.

In [23]:
function MSE_4(X, query, top_k)
    
    result = sort(MSE_2(X[:, 1:top_k], query))
    n_features, n_examples = size(X)
    
    @inbounds @fastmath for m in top_k:n_examples
        res = zero(eltype(X))
        @simd for j in 1:n_features
            aux = (query[j] .- X[j,m])
            res += aux * aux
        end
        dist = res/n_features
        
        # see if current mse is in the top pile
        if dist < result[end]
            j = top_k 
            #revisamos la lista resultado de izq a derecha
            while dist < result[j-1]
                j = j-1
                if j == 1
                    break
                end
            end            
            result[j+1:end] .= result[j:end-1]
            result[j] = dist
         end
    end
    return result
end


MSE_4 (generic function with 1 method)

In [24]:
@benchmark MSE_4(X_tr_vecs, query, 10) 

BenchmarkTools.Trial: 149 samples with 1 evaluation.
 Range (min … max):  30.109 ms … 36.762 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     34.136 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   33.736 ms ±  1.437 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

                      ▂      ▂     ▅   ▃▅▆ ▃█▂ ▆  ▂            
  ▄▇▁▄█▄▅▁▁▁▄▁▁▅▄▄▅▄▄▄█▇▅▇█▄▅█▁▇▇▅██▇▇████▇█████▇▇█▅▁▄▁▄▁▁▁▁▄ ▄
  30.1 ms         Histogram: frequency by time        36.6 ms <

 Memory estimate: 18.64 KiB, allocs estimate: 114.

In [29]:
MSE_4(X_tr_vecs, query, 10)

10-element Vector{Float64}:
 423.6640625
 430.3984375
 465.0859375
 509.84375
 513.2578125
 523.515625
 533.1796875
 545.65625
 558.1328125
 561.4140625

In [39]:
X_tr_200k = X_tr_vecs[:,1:200_000]
@benchmark MSE_4(X_tr_200k, query, 10) 

BenchmarkTools.Trial: 735 samples with 1 evaluation.
 Range (min … max):  5.780 ms …  11.656 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     6.756 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.781 ms ± 602.044 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▁  ▂   ▂▁ ▄▂▅ ▆▇▆█▃▄█▇▅ ▅▃                                 
  ▃▅█▇▆██████████▆████████████▆▆▆▃▄▃▄▃▃▁▂▃▃▁▂▁▃▃▂▂▂▂▂▃▂▂▃▃▂▁▃ ▅
  5.78 ms         Histogram: frequency by time        8.83 ms <

 Memory estimate: 16.14 KiB, allocs estimate: 94.

In [44]:
using LoopVectorization

In [53]:
function MSE_5(X, query, top_k)
    
    result = sort(MSE_2(X[:, 1:top_k], query))
    n_features, n_examples = size(X)
    
    for m in top_k:n_examples
        res = zero(eltype(X))
        @turbo  for j in 1:n_features
            aux = (query[j] - X[j,m])
            res += aux * aux
        end
        dist = res/n_features
        
        # see if current mse is in the top pile
        if dist < result[end]
            j = top_k 
            #revisamos la lista resultado de izq a derecha
            while dist < result[j-1]
                j = j-1
                if j == 1
                    break
                end
            end            
            result[j+1:end] .= result[j:end-1]
            result[j] = dist
         end
    end
    return result
end

MSE_5 (generic function with 1 method)

In [54]:
@benchmark MSE_5(X_tr_200k, query, 10) 

BenchmarkTools.Trial: 784 samples with 1 evaluation.
 Range (min … max):  5.306 ms …   8.956 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     6.326 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.364 ms ± 532.248 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                 ▁▂▁▇▃█▆▅▇▄▃▂                                  
  ▃▄▆▃▃▃▃▃▄▄▃▅▃▇▆█████████████▇▇▇▅▂▄▃▃▃▂▂▂▂▂▂▃▂▂▂▁▂▂▂▂▂▂▂▁▂▂▁ ▃
  5.31 ms         Histogram: frequency by time        8.08 ms <

 Memory estimate: 16.14 KiB, allocs estimate: 94.

In [55]:
function jdotavx(a, b)
    s = zero(eltype(a))
    @turbo for i ∈ eachindex(a, b)
        s += a[i] * b[i]
    end
    s
end

jdotavx (generic function with 1 method)

# Distances

In [40]:
using Distances
using BenchmarkTools

In [35]:
query_mat = X_te_vecs[:,1:1];

In [36]:
 x = reshape([0.1, 0.3, -0.1], 3, 1);
pairwise(Euclidean(), x, x)

1×1 Matrix{Float64}:
 0.0

In [37]:
@benchmark R = pairwise(Euclidean(), query_mat, X_te_vecs)

BenchmarkTools.Trial: 5380 samples with 1 evaluation.
 Range (min … max):  709.139 μs …  53.472 ms  ┊ GC (min … max): 0.00% … 98.09%
 Time  (median):     858.063 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   910.383 μs ± 764.679 μs  ┊ GC (mean ± σ):  1.07% ±  1.34%

    ▆██▇▇▇█▆▆▄▄▂▁▁ ▁▁▁▁▁                                         
  ▃██████████████████████▇▇▇▆▅▄▄▄▃▃▃▂▃▂▃▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂ ▄
  709 μs           Histogram: frequency by time         1.52 ms <

 Memory estimate: 78.52 KiB, allocs estimate: 6.

In [38]:
R

LoadError: UndefVarError: R not defined

In [ ]:
@benchmark R = pairwise(Euclidean(), query_mat , X_tr_vecs)

In [ ]:
?pairwise

In [ ]:
using SIMD

function find_val_in_array_simd(x::Array{T}, val::T) where {T}
    n_simd = 64
    last_pos_simd_chunk = length(x)-n_simd
    @inbounds for i in 1:n_simd:last_pos_simd_chunk
        vec_i = vload(Vec{n_simd, T}, x, i)
        sum_equality = sum(vec_i == val)
        if sum_equality >0
            return true
        end
    end

    @inbounds for i in last_pos_simd_chunk:length(x)
        if x[i] == val
            return true
        end
    end

    return false
end